# Variational AutoEncoder Generator Tensorflow
This should open a pre-trained VAE and become a Data generator

#### References:
* http://kvfrans.com/variational-autoencoders-explained/
* https://github.com/kvfrans/variational-autoencoder
* https://github.com/int8/VAE_tensorflow
* http://int8.io/variational-autoencoder-in-tensorflow/
* http://blog.fastforwardlabs.com/2016/08/22/under-the-hood-of-the-variational-autoencoder-in.html
* http://blog.fastforwardlabs.com/2016/08/12/introducing-variational-autoencoders-in-prose-and.html
* https://blog.keras.io/building-autoencoders-in-keras.html
* https://jaan.io/what-is-variational-autoencoder-vae-tutorial/
* https://arxiv.org/pdf/1606.05908.pdf
* https://arxiv.org/pdf/1312.6114.pdf
* http://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/
* https://www.tensorflow.org/get_started/embedding_viz
* https://www.youtube.com/watch?v=eBbEDRsCmv4
* https://www.youtube.com/watch?v=bbOFvxbMIV0
* https://www.youtube.com/watch?v=P78QYjWh5sM
* https://github.com/normanheckscher/mnist-tensorboard-embeddings
* http://projector.tensorflow.org/

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from pylab import *
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)

In [2]:
from models import VAE_Segnet_Generator
model = VAE_Segnet_Generator(latent_size = 40)
model_in = model.input
model_out = model.output

In [24]:
# Select only the convs
list_params = [v for v in tf.global_variables() if "z_matrix" in v.name or "conv" in v.name or "Batch" in v.name]
#list_params = [v for v in tf.global_variables() if "z_matrix" in v.name or "conv" in v.name]

In [31]:
# Problem inside the model the Batchnorm variables are BatchNorm_[1...7] but on the file 
# they are BatchNorm_7...13
# A possible solution is to give to the saver object a dictionary
# https://www.tensorflow.org/api_docs/python/tf/train/Saver
# https://stackoverflow.com/questions/39740871/how-to-save-a-dictionary-of-variables-in-tensorflow-using-the-saver-object
# dictLoadVariabel['Feature_Extractor/BatchNorm'+str(i)+'/beta'] = [tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Feature_Extractor/BatchNorm'+str(i)+'/BatchNorm/beta:0')[0]]

list_params[0].name

'LATENT/z_matrix/weights:0'

In [81]:
#tf.global_variables()
dict_match = {}
#dict_match[list_params[0].name] = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='LATENT/z_matrix/weights')[0]
# dict['stuff_on_checkpoint]=stuff_on_model

#tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='LATENT/z_matrix')
# If this tf.get_collection return more than one element you will have the "slice not slice issue"
dict_match['LATENT/z_matrix/weights'] = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='LATENT/z_matrix/weights:0')[0]


In [87]:
# The tf.collection will search (on your graph) for some scope so it could return more than one thing....
# Example this will return buch of stuff
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='DECODER/')

[<tf.Variable 'DECODER/dconv1/weights:0' shape=(3, 3, 32, 32) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv1/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv2/weights:0' shape=(3, 3, 32, 32) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv2/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv3/weights:0' shape=(3, 3, 32, 32) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv3/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv4/weights:0' shape=(3, 3, 64, 32) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv4/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv5/weights:0' shape=(5, 5, 128, 64) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv5/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv6/weights:0' shape=(5, 5, 64, 128) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv6/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv7/weights:0' shape=(5, 5, 3, 64) dtype=float32_ref>,
 <tf

### Define saver objects to load the VAE generator variables

In [82]:
# Define the saver object to load only the conv variables
#saver_load_autoencoder = tf.train.Saver(var_list=list_params)
saver_load_autoencoder = tf.train.Saver(var_list=dict_match)

ValueError: Slices must all be slices: <tf.Variable 'LATENT/z_matrix/weights:0' shape=(40, 2592) dtype=float32_ref>

### Build Graph and create session

In [80]:
init = tf.global_variables_initializer()

# Avoid allocating the whole memory
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.1)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

#sess = tf.Session()
sess.run(init)

# Initialize all random variables (Weights/Bias)
sess.run(tf.global_variables_initializer())

# Restore only the weights (From AutoEncoder)
saver_load_autoencoder.restore(sess, "/home/leoara01/work/Variational_AutoEncoder/src/notebooks/tensorflow/save/model-309")

INFO:tensorflow:Restoring parameters from /home/leoara01/work/Variational_AutoEncoder/src/notebooks/tensorflow/save/model-309


### Generate some input from the latent space

In [ ]:
# Create a latent as a normal distribution with size 20, mean 0, variance 1
latent = np.random.normal(0, 1, [1,20])

#latent = np.ones([1,20])
#latent[0,1:10] = -1

out_img = sess.run(model_out, feed_dict={model_in:latent})
plt.imshow(out_img.reshape([28,28]))
plt.show()

### Generate a video

In [ ]:
dpi = 50

def ani_frame():
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    im = ax.imshow(np.random.rand(28,28),cmap='gray',interpolation='nearest')
    #latent = np.random.normal(0, 1, [1,20])
    #out_img = sess.run(model_out, feed_dict={model_in:latent})
    #im = ax.imshow(out_img.reshape([28,28]),cmap='gray',interpolation='nearest')
    
    im.set_clim([0,1])
    fig.set_size_inches([5,5])


    tight_layout()


    def update_img(n):
        #tmp = rand(300,300)
        #im.set_data(tmp)
        latent = np.random.normal(0, 1, [1,20])
        out_img = sess.run(model_out, feed_dict={model_in:latent})
        im.set_data(out_img.reshape([28,28]))
        return im

    #legend(loc=0)
    ani = animation.FuncAnimation(fig,update_img,300,interval=30)
    writer = animation.writers['ffmpeg'](fps=30)

    ani.save('demo.mp4',writer=writer,dpi=dpi)
    return ani

In [ ]:
ani_frame()